Notebook này dùng ds101 là đủ

Dự kiến sẽ viết thành .py

TODO:
- Viết hàm download
- Xử lí Duplicate (có thể dùng pandas)

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import urllib3
import os
import time
import requests
from pathlib import Path

In [58]:
#download link using requests
def downloadSong(save_location, url_list, delay=5, last_index=0):
    success = 0
    duplicates = 0
    for link in url_list:
        file_name = link.split('/')[-1].replace("%20","")

        #print("Downloading file:%s"%file_name)  
        r = requests.get(link, stream = True)  

        #check if file is exist
        if(Path(save_location+file_name).is_file()):
            #print("Song is already exist")
            duplicates += 1
        else:
            # download started  
            with open(save_location+file_name, 'wb') as f:  
                for chunk in r.iter_content(chunk_size = 1024*1024):  
                    if chunk:  
                        f.write(chunk)
            #print("%s downloaded!\n"%file_name)
            success += 1
        time.sleep(delay)

    print("Success: {}/{}".format(success,len(url_list)))
    print("Duplicate: {}/{}".format(duplicates,len(url_list)))

In [4]:
def getSongLinks(country, tab, start_page=1, end_page=20):
    '''
    Trả về danh sách link tới các bài hát cụ thể để nghe nhạc.
    
    Tham số:
        @country: Thể loại nhạc theo tên quốc gia
        @tab: Tab của trang web: album-2021, album-moi, bai-hat-moi, vua-download
        @start_page: Giới hạn bên trái của trang thể hiện bài hát (def=1)
        @end_page: Giới hạn bên phải của trang thể hiện bài hát (def=20)
    
    Lưu ý:
        Với mỗi tab `vua-download` và `bai-hat-moi`
        sẽ chỉ crawl được trang thuộc [1; 19].
        Trang 19+ không crawl được, đừng để bị số lượng trang lừa ¯_(ツ)_/¯
    
    '''
    driver = webdriver.Chrome("chromedriver_win32/chromedriver") #
    URL = "https://www.chiasenhac.vn/mp3/" + country + ".html?tab=" + tab
    arr = np.arange(start_page, end_page)
    save_path = 'data/'
    link_list = []

    for i in arr:
        driver.get(URL + "&page=" + str(i))
        main = driver.find_element_by_class_name('content-current')
        songs = main.find_elements_by_css_selector('.media.align-items-stretch.not')
        for song in songs:
            a = song.find_element_by_tag_name("a")
            link = a.get_attribute("href")
            link_list.append(link)

    driver.close()
    return link_list

In [36]:
def getSongInfo(link_list):
    '''
    Trả về DataFrame danh sách các bài hát:
    1. Title
    2. Artist
    3. Link bài hát
    4. Link .m4a
    5. Link .mp3
    '''
    df_headers = ["Title", "Artist", "Link bài hát", "Link .m4a", "Link .mp3"]
    song_info_list = []
    
    for url in link_list:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        # lấy Title và Artist
        name = soup.find(class_="d-flex justify-content-between mb-3 box1 music-listen-title")
        if name is None:
            print("error in finding name: ", url)
            continue
        name = name.h1.text.split(" - ")
        title = name[0]
        artist = name[1]

        # lấy link tải m4a và mp3
        download_div = soup.find(id="pills-download")
        download_items = download_div.find_all("a", class_="download_item", href=True)
        m4a_link = download_items[0]['href']
        mp3_link = download_items[1]['href']

        # Thêm bài hát vào danh sách
        song_info_list.append((title, artist, url, m4a_link, mp3_link))
        
    return pd.DataFrame(song_info_list, columns=df_headers)

## Điều chỉnh thông số tại đây

In [46]:
country = "us-uk"
tab = "bai-hat-moi" # vua-download
csv_path = "data/{}_{}.csv".format(country,tab)
save_location = "songs/"

In [47]:
# Lấy link
link_list = getSongLinks(country, tab)

In [50]:
print("Số bài hát: " + str(len(link_list)))

Số bài hát: 380


In [51]:
# Trích xuất info thanh dataframe
df = getSongInfo(link_list)

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Title         380 non-null    object
 1   Artist        380 non-null    object
 2   Link bài hát  380 non-null    object
 3   Link .m4a     380 non-null    object
 4   Link .mp3     380 non-null    object
dtypes: object(5)
memory usage: 15.0+ KB


In [53]:
# Xóa phần tử trùng theo: Title && Artist
df_new = df.drop_duplicates(subset=["Title","Artist"])

In [54]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 377 entries, 0 to 379
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Title         377 non-null    object
 1   Artist        377 non-null    object
 2   Link bài hát  377 non-null    object
 3   Link .m4a     377 non-null    object
 4   Link .mp3     377 non-null    object
dtypes: object(5)
memory usage: 17.7+ KB


In [55]:
# Viết info vào trong .csv ('w': overwrite, 'a': append)
df_new.to_csv(csv_path, index=False, mode='w')

In [56]:
url_list = pd.read_csv(csv_path)["Link .m4a"]

In [ ]:
downloadSong(save_location, url_list)

In [24]:
df = pd.DataFrame(link_list, columns=["Link bài hát"])

In [35]:
df.drop([index for index in range(0,11)], inplace=True)

In [27]:
df_usuk = pd.read_csv("data/us-uk_bai-hat-moi.csv", usecols=["Link bài hát"])

In [15]:
df = df.append(df_usuk).drop_duplicates()

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 759 entries, 0 to 378
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Link bài hát  759 non-null    object
dtypes: object(1)
memory usage: 11.9+ KB


In [30]:
link_list = df_usuk["Link bài hát"].tolist()

In [ ]:
link_list

In [ ]:
for i in range(len(url_list)):
    print(i/len(url_list)*100," percent complete         \r",)

In [34]:
url = link_list[0]

page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
# lấy Title và Artist
name = soup.find(class_="d-flex justify-content-between mb-3 box1 music-listen-title")
name = name.h1.text.split(" - ")
title = name[0]
artist = name[1]

In [35]:
name

['Get Back (Extended Mix)', 'Breathe Carolina; SMACK; TITUS']